In [1]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive/My Drive/[2024-2025] AN2DL/Lecture 5

In [2]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np
import pandas as pd

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import keras_cv

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import requests
from io import BytesIO
# import cv2
# from PIL import Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

import albumentations as A
from tensorflow.keras.saving import register_keras_serializable

2.16.1


In [3]:
data = np.load('/kaggle/input/data-mars2/augmented_dataset_1.npz')
print(data.files)

['X_train', 'X_val', 'X_test', 'X_to_predict']


In [4]:
#X_train = data['X_train']
#y_train = data['y_train']
#X_val = data['X_val']
#y_val = data['y_val']
#X_test = data['X_test']
#y_test = data['y_test']
#X_to_predict = data['X_to_predict']
#class_weights = data['class_weights']

#print("Shape of TRAIN:", " X",X_train.shape, " y",y_train.shape)
#print("Shape of VAL:", " X",X_val.shape, " y",y_val.shape)
#print("Shape of TEST:", " X",X_test.shape, " y",y_test.shape)
#print("Shape of X_to_predict:", X_to_predict.shape)

#targets = ['Background', 'Soil', 'Bedrock', 'Sand', 'Big Rock']
#NUM_CLASSES = len(targets)
data = np.load('/kaggle/input/data-mars/augmented_dataset-3.npz')
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
X_to_predict = data['X_to_predict']

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of X_to_predict:", X_to_predict.shape)

targets = ['Background', 'Soil', 'Bedrock', 'Sand', 'Big Rock']
NUM_CLASSES = len(targets)

Shape of X_train: (50100, 2, 64, 128)
Shape of X_val: (6250, 2, 64, 128)
Shape of X_test: (6275, 2, 64, 128)
Shape of X_to_predict: (10022, 64, 128)


In [5]:
#def np_to_tf(X, y, batch_size = 64, shuffle_buffer_size=1000):
#    X = X.astype(np.uint8)
#    y = y.astype(np.uint8)
#    
#    dataset = tf.data.Dataset.from_tensor_slices((X, y))

#    dataset = dataset.shuffle(buffer_size=shuffle_buffer_size)
    
#    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
#    print(dataset.element_spec)
#    print(f"Num of batchs: {len(dataset)}, Num of images: {len(dataset)*batch_size}")

#    return dataset

#X_train_tf = np_to_tf(X_train, y_train)
#X_val_tf = np_to_tf(X_val, y_val)
#X_test_tf = np_to_tf(X_test, y_test)
def np_to_tf(X, batch_size = 64):
    # Split the array into images and masks
    X_images = X[:, 0]  # Grayscale images 
    X_masks = X[:, 1]   # Masks (shape: 
    
    # Reshape the images to 3 channels (by repeating the grayscale channel)
    X_images_rgb = np.repeat(X_images[..., np.newaxis], 3, axis=-1)  
    
    # Reshape the masks to have a single channel
    X_masks = X_masks[..., np.newaxis]  
    
    # Ensure images have dtype float32 and masks have dtype int32
    X_images_rgb = X_images_rgb.astype(np.uint8)
    X_masks = X_masks.astype(np.uint8)
    
    # Create a TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((X_images_rgb, X_masks))
    
    # Prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    # Print dataset element spec to verify the shape
    print(dataset.element_spec)
    print(f"Num of batchs: {len(dataset)}, Num of images: {len(dataset)*batch_size}")

    return dataset

X_train_tf = np_to_tf(X_train)
X_val_tf = np_to_tf(X_val)
X_test_tf = np_to_tf(X_test)

(TensorSpec(shape=(None, 64, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 64, 128, 1), dtype=tf.uint8, name=None))
Num of batchs: 783, Num of images: 50112
(TensorSpec(shape=(None, 64, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 64, 128, 1), dtype=tf.uint8, name=None))
Num of batchs: 98, Num of images: 6272
(TensorSpec(shape=(None, 64, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 64, 128, 1), dtype=tf.uint8, name=None))
Num of batchs: 99, Num of images: 6336


In [6]:
#def np_to_tf_no_mask(X, batch_size = 64):
#    X_images = X  
    
#    X_images_rgb = np.repeat(X_images[..., np.newaxis], 3, axis=-1)  # (20920, 64, 128, 3)
    
#    X_images_rgb = X_images_rgb.astype(np.uint8)
    
#    dataset = tf.data.Dataset.from_tensor_slices(X_images_rgb)
    
#    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
#    print(dataset.element_spec)
#    print(f"Num of batchs: {len(dataset)}, Num of images: {len(dataset)*batch_size}")

#    return dataset

#X_to_predict_tf = np_to_tf_no_mask(X_to_predict)
def np_to_tf_no_mask(X, batch_size = 64):
    # Split the array into images and masks
    X_images = X  # Grayscale images 
    
    # Reshape the images to 3 channels (by repeating the grayscale channel)
    X_images_rgb = np.repeat(X_images[..., np.newaxis], 3, axis=-1)  
    
    # Reshape the masks to have a single channel
    
    # Ensure images have dtype float32 and masks have dtype int32
    X_images_rgb = X_images_rgb.astype(np.uint8)
    
    # Create a TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices(X_images_rgb)
    
    # Prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    # Print dataset element spec to verify the shape
    print(dataset.element_spec)
    print(f"Num of batchs: {len(dataset)}, Num of images: {len(dataset)*batch_size}")

    return dataset

X_to_predict_tf = np_to_tf_no_mask(X_to_predict)

TensorSpec(shape=(None, 64, 128, 3), dtype=tf.uint8, name=None)
Num of batchs: 157, Num of images: 10048


In [7]:
# Set learning rate for the optimiser
LEARNING_RATE = 1e-4

# Set early stopping patience threshold
PATIENCE = 20

# Set maximum number of training epochs
EPOCHS = 200

In [8]:
# Define custom Mean Intersection Over Union metric
@register_keras_serializable()
class MeanIntersectionOverUnion(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes, labels_to_exclude=None, name="mean_iou", dtype=None):
        super(MeanIntersectionOverUnion, self).__init__(num_classes=num_classes, name=name, dtype=dtype)
        if labels_to_exclude is None:
            labels_to_exclude = [0]  # Default to excluding label 0
        self.labels_to_exclude = labels_to_exclude

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert predictions to class labels
        y_pred = tf.math.argmax(y_pred, axis=-1)

        # Flatten the tensors
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(y_pred, [-1])

        # Apply mask to exclude specified labels
        for label in self.labels_to_exclude:
            mask = tf.not_equal(y_true, label)
            y_true = tf.boolean_mask(y_true, mask)
            y_pred = tf.boolean_mask(y_pred, mask)

        # Update the state
        return super().update_state(y_true, y_pred, sample_weight)

def unet_block(input_tensor, filters, kernel_size=3, activation='relu', stack=2, dropout_rate = 0.3, name=''):
    # Initialise the input tensor
    x = input_tensor

    # Apply a sequence of Conv2D, Batch Normalisation, and Activation layers for the specified number of stacks
    for i in range(stack):
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', name=name + 'conv' + str(i + 1))(x)
        x = tfkl.BatchNormalization(name=name + 'bn' + str(i + 1))(x)
        x = tfkl.Activation(activation, name=name + 'activation' + str(i + 1))(x)

    # if x.shape[-1] != input_tensor.shape[-1]:  # If number of channels don't match
       # input_tensor = tfkl.Conv2D(filters, kernel_size=1, padding='same', name=name + 'conv_1x1_input')(input_tensor)

    # x = tfkl.Add()([x, input_tensor]) # add residual connections
    # Return the transformed tensor
    return x


In [9]:
class NCutLoss2D(tf.keras.losses.Loss):
    def __init__(self, radius=4, sigma_1=5.0, sigma_2=1.0, name="NCutLoss2D"):
        super().__init__(name=name)
        self.radius = radius
        self.sigma_1 = sigma_1
        self.sigma_2 = sigma_2

    def gaussian_kernel(self, radius, sigma):
        """
        Generate a 2D Gaussian kernel.
        """
        size = 2 * radius + 1
        x, y = np.meshgrid(np.arange(size) - radius, np.arange(size) - radius)
        kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
        kernel /= np.sum(kernel)
        return tf.convert_to_tensor(kernel, dtype=tf.float32)

    def call(self, y_true, y_pred):
        """
        Compute the continuous N-Cut loss.
        :param y_true: Ground truth labels (sparse integer labels).
        :param y_pred: Predicted class probabilities.
        :return: Continuous N-Cut loss.
        """
        # Ensure y_true matches the shape of y_pred
        y_true_resized = tf.image.resize(y_true, tf.shape(y_pred)[1:3], method='nearest')
        y_true_resized = tf.cast(tf.round(y_true_resized), tf.int32)

        # Convert sparse labels to one-hot format
        num_classes = tf.shape(y_pred)[-1]
        y_true_one_hot = tf.one_hot(y_true_resized, num_classes)
        y_true_one_hot = tf.cast(y_true_one_hot, tf.float32)

        kernel = self.gaussian_kernel(self.radius, self.sigma_1)
        kernel = tf.reshape(kernel, [2 * self.radius + 1, 2 * self.radius + 1, 1, 1])

        def compute_class_loss(k):
            """
            Compute the loss for a single class.
            """
            class_probs = y_pred[..., k]  # Shape: (batch_size, height, width)
            class_probs = tf.expand_dims(class_probs, axis=-1)  # Shape: (batch_size, height, width, 1)
            class_true = y_true_one_hot[..., k]  # Shape: (batch_size, height, width)
            #class_true = tf.expand_dims(class_true, axis=-1)  # Shape: (batch_size, height, width, 1)

            # Class mean pixel value
            class_mean = tf.reduce_sum(class_probs * class_true, axis=(1, 2), keepdims=True) / (
                tf.reduce_sum(class_probs, axis=(1, 2), keepdims=True) + 1e-5
            )

            # Difference from class mean
            diff = tf.reduce_sum((class_true - class_mean) ** 2, axis=-1, keepdims=True)

            # Compute weights
            weights = tf.exp(-diff ** 2 / (self.sigma_2 ** 2))

            # Compute N-cut numerator and denominator
            spatially_smoothed_probs = tf.nn.depthwise_conv2d(
                class_probs * weights, kernel, strides=[1, 1, 1, 1], padding="SAME"
            )
            numerator = tf.reduce_sum(class_probs * spatially_smoothed_probs)
            denominator = tf.reduce_sum(class_probs * tf.nn.depthwise_conv2d(weights, kernel, strides=[1, 1, 1, 1], padding="SAME"))

            return tf.abs(numerator / (denominator + 1e-6))

        # Use tf.map_fn to iterate over classes
        class_losses = tf.map_fn(compute_class_loss, tf.range(num_classes), dtype=tf.float32)
        total_loss = tf.cast(num_classes, tf.float32) - tf.reduce_sum(class_losses)
        return total_loss

def combined_loss(y_true, y_pred):
    # Loss U-Net 1: focalizzata sull'output intermedio (esempio MSE)
    n_cut_loss = NCutLoss2D(radius=4, sigma_1=5.0, sigma_2=1.0)
    loss_unet1 = n_cut_loss(y_true - y_pred)
    # Loss U-Net 2: ricostruzione finale
    loss_unet2 = tf.reduce_mean(tf.square(y_true - y_pred))
    return 0.5 * loss_unet1 + 0.5 * loss_unet2

def dice_loss_multiclass_excluding_background(y_true, y_pred, smooth=1e-6):
    # One-hot encode y_true to match the shape of y_pred
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])  # (batch, H, W, num_classes)

    # Exclude background (assumed to be class 0)
    y_true = y_true[..., 1:]  # Exclude channel 0
    y_pred = y_pred[..., 1:]  # Exclude channel 0

    # Reshape to (batch_size * image_size, num_classes)
    y_true = tf.reshape(y_true, (-1, tf.shape(y_pred)[-1]))
    y_pred = tf.reshape(y_pred, (-1, tf.shape(y_pred)[-1]))

    # Compute Dice coefficient for each class
    intersection = tf.reduce_sum(y_true * y_pred, axis=0)
    union = tf.reduce_sum(y_true, axis=0) + tf.reduce_sum(y_pred, axis=0)
    dice = (2. * intersection + smooth) / (union + smooth)

    # Average Dice Loss across all non-background classes
    return 1 - tf.reduce_mean(dice)

def dice_loss_multiclass_with_background_weight(y_true, y_pred, background_weight=0.001, other_class_weight=0.05, smooth=1e-6):
    # One-hot encode y_true to match the shape of y_pred
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])  # (batch, H, W, num_classes)

    # Reshape to (batch_size * image_size, num_classes)
    y_true = tf.reshape(y_true, (-1, tf.shape(y_pred)[-1]))
    y_pred = tf.reshape(y_pred, (-1, tf.shape(y_pred)[-1]))

    # Compute Dice coefficient for each class
    intersection = tf.reduce_sum(y_true * y_pred, axis=0)
    union = tf.reduce_sum(y_true, axis=0) + tf.reduce_sum(y_pred, axis=0)
    dice = (2. * intersection + smooth) / (union + smooth)

    # Create class-specific weights
    num_classes = tf.shape(y_pred)[-1]
    class_weights = tf.ones(num_classes)
    class_weights = tf.tensor_scatter_nd_update(
        class_weights, 
        [[0]],  # index of background class 
        [background_weight]  # new weight for background
    )

    class_weights = tf.tensor_scatter_nd_update(
        class_weights, 
        [[1]],  
        [other_class_weight]  
    )

    class_weights = tf.tensor_scatter_nd_update(
        class_weights, 
        [[2]],  
        [other_class_weight]  
    )

    class_weights = tf.tensor_scatter_nd_update(
        class_weights, 
        [[3]],  
        [other_class_weight]  
    )

    # Apply class weights to Dice coefficients
    weighted_dice = dice * class_weights

    # Average Dice Loss across all classes
    return 1 - tf.reduce_mean(weighted_dice)


from scipy.ndimage import distance_transform_edt

def dice_loss_multiclass_excluding_background(y_true, y_pred, smooth=1e-6):
    # One-hot encode y_true to match the shape of y_pred
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])  # (batch, H, W, num_classes)

    # Exclude background (assumed to be class 0)
    y_true = y_true[..., 1:]  # Exclude channel 0
    y_pred = y_pred[..., 1:]  # Exclude channel 0

    # Reshape to (batch_size * image_size, num_classes)
    y_true = tf.reshape(y_true, (-1, tf.shape(y_pred)[-1]))
    y_pred = tf.reshape(y_pred, (-1, tf.shape(y_pred)[-1]))

    # Compute Dice coefficient for each class
    intersection = tf.reduce_sum(y_true * y_pred, axis=0)
    union = tf.reduce_sum(y_true, axis=0) + tf.reduce_sum(y_pred, axis=0)
    dice = (2. * intersection + smooth) / (union + smooth)

    # Average Dice Loss across all non-background classes
    return 1 - tf.reduce_mean(dice)

def categorical_focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1 - K.epsilon())
        
        cross_entropy = -y_true * tf.math.log(y_pred)
        focal_loss = alpha * (1 - y_pred) ** gamma * cross_entropy
        
        return tf.reduce_mean(tf.reduce_sum(focal_loss, axis=-1))
    
    return loss

def compute_distance_map(y_true):
    batch_size = y_true.shape[0]
    distance_maps = np.zeros_like(y_true)
    for i in range(batch_size):
        for c in range(y_true.shape[-1]):
            distance_maps[i, :, :, c] = distance_transform_edt(y_true[i, :, :, c])
    return distance_maps

def boundary_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    
    sobel_filter_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32)
    sobel_filter_x = sobel_filter_x[:, :, tf.newaxis, tf.newaxis]
    sobel_filter_y = tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=tf.float32)
    sobel_filter_y = sobel_filter_y[:, :, tf.newaxis, tf.newaxis]

    grad_x = tf.nn.conv2d(y_true, sobel_filter_x, strides=[1, 1, 1, 1], padding='SAME')
    grad_y = tf.nn.conv2d(y_true, sobel_filter_y, strides=[1, 1, 1, 1], padding='SAME')

    gradient_magnitude = tf.sqrt(tf.square(grad_x) + tf.square(grad_y))

    normalized_boundary = tf.clip_by_value(gradient_magnitude, 0, 1)

    boundary_loss_value = tf.reduce_mean(normalized_boundary * tf.abs(y_pred - y_true))

    return boundary_loss_value

def combined_loss(y_true, y_pred):
    loss_dice = dice_loss_multiclass_excluding_background(y_true, y_pred)
    loss_focal = categorical_focal_loss()(y_true, y_pred)  
    loss_boundary = boundary_loss(y_true, y_pred)
    return 0.7 * loss_dice + 0.15 * loss_focal + 0.15 * loss_boundary 


In [10]:
# 1st U-Net

def unet_block(input_tensor, filters, kernel_size=3, activation='relu', stack=2, dropout_rate = 0.3, name=''):
    # Initialise the input tensor
    x = input_tensor

    # Apply a sequence of Conv2D, Batch Normalisation, and Activation layers for the specified number of stacks
    for i in range(stack):
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', name=name + 'conv' + str(i + 1))(x)
        x = tfkl.BatchNormalization(name=name + 'bn' + str(i + 1))(x)
        x = tfkl.Activation(activation, name=name + 'activation' + str(i + 1))(x)

    # if x.shape[-1] != input_tensor.shape[-1]:  # If number of channels don't match
       # input_tensor = tfkl.Conv2D(filters, kernel_size=1, padding='same', name=name + 'conv_1x1_input')(input_tensor)
    
    # x = tfkl.Add()([x, input_tensor]) # add residual connections
    # Return the transformed tensor
    return x

def get_unet_model(input_shape=(64, 128, 3), num_classes=NUM_CLASSES, seed=seed):
    tf.random.set_seed(seed)
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Downsampling path
    down_block_1 = unet_block(input_layer, 32, name='down_block1_')
    d1 = tfkl.MaxPooling2D()(down_block_1)

    down_block_2 = unet_block(d1, 64, name='down_block2_')
    d2 = tfkl.MaxPooling2D()(down_block_2)

    down_block_3 = unet_block(d2, 128, name='down_block3_')
    d3 = tfkl.MaxPooling2D()(down_block_3)

    down_block_4 = unet_block(d3, 256, name='down_block4_')
    d4 = tfkl.MaxPooling2D()(down_block_4)

    down_block_5 = unet_block(d4, 512, name='down_block5_')
    d5 = tfkl.MaxPooling2D()(down_block_5)

    # Bottleneck
    bottleneck = unet_block(d5, 1024, name='bottleneck')
    bottleneck = tfkl.Dropout(0.5)(bottleneck)

    # Upsampling path
    u1 = tfkl.UpSampling2D()(bottleneck)
    u1 = tfkl.Concatenate()([u1, down_block_5])
    u1 = unet_block(u1, 512, name='up_block1_')
    
    u2 = tfkl.UpSampling2D()(u1)
    u2 = tfkl.Concatenate()([u2, down_block_4])
    u2 = unet_block(u2, 256, name='up_block2_')
    
    u3 = tfkl.UpSampling2D()(u2)
    u3 = tfkl.Concatenate()([u3, down_block_3])
    u3 = unet_block(u3, 128, name='up_block3_')
    
    u4 = tfkl.UpSampling2D()(u3)
    u4 = tfkl.Concatenate()([u4, down_block_2])
    u4 = unet_block(u4, 64, name='up_block4_')
    
    u5 = tfkl.UpSampling2D()(u4)
    u5 = tfkl.Concatenate()([u5, down_block_1])
    u5 = unet_block(u5, 32, name='up_block5_')

    # Output Layer
    output_layer = tfkl.Conv2D(num_classes, kernel_size=1, padding='same', activation="softmax", name='output_layer')(u5)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model


In [11]:
from tensorflow.keras import Model

def build_wnet(input_shape=(64, 128, 3)):
    # First U-Net
    unet1 = get_unet_model()
    
    # Second U-Net
    unet2 = get_unet_model()

    # Input 
    inputs = tf.keras.Input(input_shape)
    
    unet1_output = unet1(inputs)
    
    # Adjusting channels
    adjusted_output = tfkl.Conv2D(3, kernel_size=(1, 1), padding='same')(unet1_output)

    unet2_output = unet2(adjusted_output)
    
    # Combined Model
    return Model(inputs, unet2_output)

# WNet model
model = build_wnet()

In [12]:
# Print a detailed summary of the model with expanded nested layers and trainable parameters.
model.summary(expand_nested=True, show_trainable=True)

# Generate and display a graphical representation of the model architecture.
tf.keras.utils.plot_model(model, show_trainable=True, expand_nested=True, dpi=70)

# Compile the model
print("Compiling model...")
model.compile(
    #loss=dice_loss_multiclass_with_background_weight,
    loss=dice_loss_multiclass_with_background_weight,
    optimizer=tf.keras.optimizers.AdamW(LEARNING_RATE),
    metrics=["accuracy", MeanIntersectionOverUnion(num_classes=NUM_CLASSES, labels_to_exclude=[0])]
)
print("Model compiled!")

# Setup callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=PATIENCE,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.5,          
    patience=3,          
    min_lr=1e-6          
)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer (InputLayer)    │ (None, 64, 128, 3)    │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ functional_1 (Functional)   │ (None, 64, 128, 5)    │ 31,466,885 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ input_layer            │ (None, 64, 128, 3)    │          0 │   -   │
│ (InputLayer)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ down_block1_conv1      │ (None, 64, 128, 32)   │        896 │   Y   │
│ (Conv2D)                    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ down_block1_bn1        │ (None, 64, 128, 32)   │        128 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 64, 128, 32)   │          0 │   -   │
│ down_block1_activation1     │                       │            │       │
│ (Activation)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ down_block1_conv2      │ (None, 64, 128, 32)   │      9,248 │   Y   │
│ (Conv2D)                    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ down_block1_bn2        │ (None, 64, 128, 32)   │        128 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 64, 128, 32)   │          0 │   -   │
│ down_block1_activation2     │                       │            │       │
│ (Activation)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ max_pooling2d          │ (None, 32, 64, 32)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ down_block2_conv1      │ (None, 32, 64, 64)    │     18,496 │   Y   │
│ (Conv2D)                    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ down_block2_bn1        │ (None, 32, 64, 64)    │        256 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 32, 64, 64)    │          0 │   -   │
│ down_block2_activation1     │                       │            │       │
│ (Activation)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ down_block2_conv2      │ (None, 32, 64, 64)    │     36,928 │   Y   │
│ (Conv2D)                    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ down_block2_bn2        │ (None, 32, 64, 64)    │        256 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 32, 64, 64)    │          0 │   -   │
│ down_block2_activation2     │                       │            │       │
│ (Activation)                │                       │            │     

 Total params: 62,933,788 (240.07 MB)

 Trainable params: 62,909,724 (239.98 MB)

 Non-trainable params: 24,064 (94.00 KB)

Compiling model...
Model compiled!


In [13]:
# Train the all model
history = model.fit(
    X_train_tf,
    epochs=EPOCHS,
    validation_data=X_val_tf,
    #class_weight = class_weights_dict,
    callbacks=[early_stopping, reduce_lr],
    verbose=2
).history

# Calculate and print the final validation accuracy
final_val_meanIoU = round(max(history['val_mean_iou'])* 100, 2)
print(f'Final validation Mean Intersection Over Union: {final_val_meanIoU}%')

Epoch 1/200


I0000 00:00:1734098665.723161      66 service.cc:145] XLA service 0x78fd24002c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734098665.723218      66 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1734098700.803389      66 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


783/783 - 272s - 348ms/step - accuracy: 0.5138 - loss: 0.9794 - mean_iou: 0.1507 - val_accuracy: 0.4053 - val_loss: 0.9844 - val_mean_iou: 0.2521 - learning_rate: 1.0000e-04
Epoch 2/200
783/783 - 163s - 209ms/step - accuracy: 0.5710 - loss: 0.9688 - mean_iou: 0.1886 - val_accuracy: 0.5454 - val_loss: 0.9808 - val_mean_iou: 0.3638 - learning_rate: 1.0000e-04
Epoch 3/200
783/783 - 163s - 208ms/step - accuracy: 0.5649 - loss: 0.9629 - mean_iou: 0.1896 - val_accuracy: 0.5156 - val_loss: 0.9707 - val_mean_iou: 0.3511 - learning_rate: 1.0000e-04
Epoch 4/200
783/783 - 163s - 209ms/step - accuracy: 0.5824 - loss: 0.9578 - mean_iou: 0.2061 - val_accuracy: 0.4386 - val_loss: 0.9778 - val_mean_iou: 0.2680 - learning_rate: 1.0000e-04
Epoch 5/200
783/783 - 163s - 209ms/step - accuracy: 0.5843 - loss: 0.9523 - mean_iou: 0.2140 - val_accuracy: 0.5366 - val_loss: 0.9776 - val_mean_iou: 0.4443 - learning_rate: 1.0000e-04
Epoch 6/200
783/783 - 163s - 209ms/step - accuracy: 0.5857 - loss: 0.9538 - mean_i

In [14]:
model_filename = 'UNet_'+str(final_val_meanIoU)+'.keras'
model.save(model_filename)

In [15]:
test_loss, test_accuracy, test_mean_iou = model.evaluate(X_test_tf)

# Print the results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Mean IoU: {test_mean_iou}")

99/99 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.6462 - loss: 0.9775 - mean_iou: 0.5643
Test Loss: 0.9776154160499573
Test Accuracy: 0.6390791535377502
Test Mean IoU: 0.5584849715232849


In [16]:
y_preds = model.predict(X_to_predict_tf)
y_preds = np.argmax(y_preds, axis=-1)
print(f"Predictions shape: {y_preds.shape}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 15s 81ms/step
Predictions shape: (10022, 64, 128)


In [17]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [18]:
# Create and download the csv submission file
timestep_str = f'UNet_{test_mean_iou}'
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(y_preds)
submission_df.to_csv(submission_filename, index=False)